# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

/Users/aguigal/.zshenv:1: permission denied: /Library/Java/JavaVirtualMachines/jdk1.8.0_271.jdk/Contents/Home
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_hands = mp.solutions.hands


In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections


In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                         )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [27]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [45]:
len(results.pose_landmarks.landmark)

33

In [47]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
results.pose_landmarks.landmark[0]

x: 0.4473770558834076
y: -0.27084121108055115
z: -0.7756742238998413
visibility: 0.9848765134811401

In [19]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [19]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [20]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()

    if results.face_landmarks
    else np.zeros(1404)

IndentationError: unexpected indent (1384755025.py, line 3)

In [6]:
def extract_keypoints(results):
    pose, face, lh, rh = None, None, None, None
    if results.pose_landmarks:
        ref = results.pose_landmarks.landmark[0]
        pose = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z, res.visibility] for res in
             results.pose_landmarks.landmark]
        ).flatten() if results.pose_landmarks else np.zeros(33 * 4)
        face = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.face_landmarks.landmark]
        ).flatten() if results.face_landmarks else np.zeros(468 * 3)
        lh = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.left_hand_landmarks.landmark]
        ).flatten() if results.left_hand_landmarks else np.zeros(
            21 * 3)
        rh = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.right_hand_landmarks.landmark]
        ).flatten() if results.right_hand_landmarks else np.zeros(
            21 * 3)
    else:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                         results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
        face = np.array([[res.x, res.y, res.z] for res in
                         results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
        lh = np.array([[res.x, res.y, res.z] for res in
                       results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(
            21 * 3)
        rh = np.array([[res.x, res.y, res.z] for res in
                       results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(
            21 * 3)
    return np.concatenate([pose, face, lh, rh])

In [22]:
result_test = extract_keypoints(results)

In [23]:
result_test

array([ 0.67701447,  0.29765448, -1.79628587, ...,  0.50258791,
        1.30048299, -0.08565646])

In [14]:
np.save('0', result_test)

NameError: name 'result_test' is not defined

In [15]:
np.load('0.npy')

array([ 0.60055482,  0.82639527, -2.14327669, ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA2')

# Actions that we try to detect
actions = np.array(['hello', 'iloveyou', 'thanks'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [8]:
DATA_PATH

'MP_DATA2'

In [8]:
for action in actions:
    if np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int).size != 0:
        dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    else:
        dirmax = 0
    for sequence in range(1, no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [14]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)

                # Change referential
                ## TODO::



                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [15]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils     import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'hello': 0, 'iloveyou': 1, 'thanks': 2}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

KeyboardInterrupt: 

In [16]:
np.array(sequences).shape

NameError: name 'sequences' is not defined

In [21]:
np.array(labels).shape

(90,)

In [22]:
X = np.array(sequences)

In [23]:
X.shape

(90, 30, 1662)

In [24]:
y = to_categorical(labels).astype(int)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [26]:
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 193ms/step - loss: 1.4432 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 60ms/step - loss: 6.2594 - categorical_accuracy: 0.3176
Epoch 3/2000
3/3 [==============================] - 0s 57ms/step - loss: 52.3391 - categorical_accuracy: 0.3176
Epoch 4/2000
3/3 [==============================] - 0s 54ms/step - loss: 23.9040 - categorical_accuracy: 0.2706
Epoch 5/2000
3/3 [==============================] - 0s 62ms/step - loss: 16.6260 - categorical_accuracy: 0.3412
Epoch 6/2000
3/3 [==============================] - 0s 52ms/step - loss: 20.0651 - categorical_accuracy: 0.2941
Epoch 7/2000
3/3 [==============================] - 0s 56ms/step - loss: 10.6397 - categorical_accuracy: 0.4588
Epoch 8/2000
3/3 [==============================] - 0s 54ms/step - loss: 4.3573 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 53ms/step - loss: 3.6167 - categorical_accuracy: 0.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [15]:
res = model.predict(X_test)

NameError: name 'X_test' is not defined

In [80]:
actions[np.argmax(res[4])]

'iloveyou'

In [81]:
actions[np.argmax(y_test[4])]

'iloveyou'

# 9. Save Weights

In [37]:
model.save('action-test_reference.h5')

In [42]:
del model

In [13]:
model.load_weights('action-test_reference.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [82]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [83]:
yhat = model.predict(X_test)

In [84]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [85]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [86]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [37]:
from scipy import stats

In [21]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [57]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1296x1296 with 0 Axes>

In [28]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        #compute for different frame length
        predict = False
        upper = False
        prob_lowerbound, prob_upperbound, prob_middle = 0,0,0
        pos_lowerbound, pos_upperbound, pos_middle = -1, -1, -1
        res_lowerbound, res_upperbound, res_middle = None, None, None
        pos_prediction = -1
        prob_prediction = 0
        res = None
        sequence_tmp_lowerbound = sequence[-10:]
        if len(sequence_tmp_lowerbound) == 10:
            predict = True
            res_lowerbound = model.predict(np.expand_dims(sequence_tmp_lowerbound, axis=0))[0]
            pos_lowerbound = np.argmax(res_lowerbound)
            prob_lowerbound = res_lowerbound[pos_lowerbound]

        sequence_tmp_upperbound = sequence[-50:]
        if len(sequence_tmp_upperbound) == 50:
            predict = True
            upper = True
            res_upperbound = model.predict(np.expand_dims(sequence_tmp_upperbound, axis=0))[0]
            pos_upperbound = np.argmax(res_upperbound)
            prob_upperbound = res_upperbound[pos_upperbound]

        sequence_tmp_midle = sequence[-30:]
        if len(sequence_tmp_midle) == 30:
            predict = True
            res_middle = model.predict(np.expand_dims(sequence_tmp_midle, axis=0))[0]
            pos_middle = np.argmax(res_middle)
            prob_middle = res_middle[pos_middle]

        #compare
        if prob_middle >= prob_upperbound and prob_middle >= prob_lowerbound:
            #print("30")
            prob_prediction = prob_middle
            pos_prediction = pos_middle
            res = res_middle
            if upper:
                sequence = sequence[-30:]
        elif prob_upperbound >= prob_middle and prob_upperbound>= prob_lowerbound:
            #print("50")
            prob_prediction = prob_upperbound
            pos_prediction = pos_upperbound
            res = res_upperbound
            if upper:
                sequence = sequence[-50:]
        else:
            #print("10")
            prob_prediction = prob_lowerbound
            pos_prediction = pos_lowerbound
            res = res_lowerbound
            if upper:
                sequence = sequence[-10:]


        if predict:
            #print(model.predict(np.expand_dims(sequence, axis=0))[0])
            #res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(pos_prediction)


        #3. Viz logic
            if np.unique(predictions[-10:])[0]==pos_prediction:
                if prob_prediction > threshold:
                    #print(actions[pos_prediction])
                    if len(sentence) > 0:
                        if actions[pos_prediction] != sentence[-1]: #add if different
                            sentence.append(actions[pos_prediction])
                    else:
                        sentence.append(actions[pos_prediction])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 